**Cloning Git repository**

In [1]:
!git clone https://github.com/Adu3108/Binarized_VAE.git

Cloning into 'Binarized_VAE'...
remote: Enumerating objects: 9277, done.
remote: Counting objects: 100% (3181/3181), done.
remote: Compressing objects: 100% (3171/3171), done.
remote: Total 9277 (delta 8), reused 3180 (delta 8), pack-reused 6096
Receiving objects: 100% (9277/9277), 147.00 MiB | 16.63 MiB/s, done.
Resolving deltas: 100% (11/11), done.
Checking out files: 100% (9157/9157), done.


**Extracting the necessary code**

In [2]:
!mv /content/Binarized_VAE/Second\ VAE/Normal\ Version/vae.py /content/

**Install all the dependencies**

In [3]:
!pip install torch 
!pip install torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Install Dataset**

**MNIST Dataset**

In [ ]:
!mv /content/Binarized_VAE/Datasets/MNIST\ Dataset\ JPG\ format.zip /content/ 

**Preprocessing of Dataset**

There will be 2 directories :- 
1.   Training
2.   Testing

Each directory consists of 10 classes named 0,1,2,...,9

Each class contains 10 images

In [ ]:
import os

os.rename('/content/MNIST Dataset JPG format.zip','MNIST.zip')

In [ ]:
!jar xvf /content/MNIST.zip

In [ ]:
os.rename('/content/MNIST Dataset JPG format', 'MNIST')
os.rename('/content/MNIST/MNIST - JPG - training','/content/MNIST/Training')
os.rename('/content/MNIST/MNIST - JPG - testing', '/content/MNIST/Testing')
os.mkdir("/content/MNIST/Validation")
for i in range(10):
  path = os.path.join('/content/MNIST/Validation',str(i))
  os.mkdir(path)

In [ ]:
os.chdir('/content/MNIST/Training')
arr = os.listdir()
for i in arr:
  tmp = os.path.join('/content/MNIST/Training',i)
  os.chdir(tmp)
  val = os.listdir()[10:20]
  rem = os.listdir()[20:]
  for j in val:
    original_path = os.path.join(tmp,j)
    val_path = os.path.join('/content/MNIST/Validation',i)
    val_path = os.path.join(val_path,j)
    os.replace(original_path,val_path)
  for j in rem:
    os.remove(j)
os.chdir('/content/MNIST/Testing')
arr = os.listdir()
for i in arr:
  tmp = os.path.join('/content/MNIST/Testing',i)
  os.chdir(tmp)
  rem = os.listdir()[10:]
  for j in rem:
    os.remove(j)
os.chdir('/content')

**Caltech-101 Dataset**

In [4]:
!mv /content/Binarized_VAE/Datasets/Caltech-101 /content/

**Creating Training, Validation, Testing dataset**

In [5]:
import os

os.mkdir('/content/Dataset')
os.mkdir('/content/Dataset/Training')
os.mkdir('/content/Dataset/Testing')
os.mkdir('/content/Dataset/Validation')
os.chdir("/content/Caltech-101/")
l = os.listdir()
os.chdir('/content/Dataset/Training')
for i in l:
  path = os.path.join('/content/Dataset/Training',i)
  os.mkdir(path)
os.chdir('/content/Dataset/Validation')
for i in l:
  path = os.path.join('/content/Dataset/Validation',i)
  os.mkdir(path)  
os.chdir('/content/Dataset/Testing')
for i in l:
  path = os.path.join('/content/Dataset/Testing',i)
  os.mkdir(path)

In [6]:
for i in l:
  path = os.path.join('/content/Caltech-101/',i)
  os.chdir(path)
  training = os.listdir()[:10]
  validation = os.listdir()[10:20]
  testing = os.listdir()[20:30]
  rem = os.listdir()[30:]
  for j in training:
    original_path = os.path.join(path,j)
    val_path = os.path.join('/content/Dataset/Training',i)
    val_path = os.path.join(val_path,j)
    os.replace(original_path,val_path)
  for j in validation:
    original_path = os.path.join(path,j)
    val_path = os.path.join('/content/Dataset/Validation',i)
    val_path = os.path.join(val_path,j)
    os.replace(original_path,val_path)
  for j in testing:
    original_path = os.path.join(path,j)
    val_path = os.path.join('/content/Dataset/Testing',i)
    val_path = os.path.join(val_path,j)
    os.replace(original_path,val_path)
  for j in rem:
    os.remove(j)
  os.chdir('/content')

**Training**

In [ ]:
os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"
!python binary_vae_v1.py train --epochs 100 --batch 1 --n_latent 30 --dimensions 120x160 --train_set /content/Dataset/Training --validation_set /content/Dataset/Validation --weights /content/weights.pt

In [11]:
%matplotlib inline
%matplotlib notebook

**Fixing the final directory structure**

In [12]:
!mkdir Results
!mkdir /content/Results/Input
!mkdir /content/Results/Reconstructed

mkdir: cannot create directory ‘Results’: File exists
mkdir: cannot create directory ‘/content/Results/Input’: File exists
mkdir: cannot create directory ‘/content/Results/Reconstructed’: File exists


In [13]:
!python vae.py calibrate --batch 1 --n_latent 30 --dimensions 120x160 --cal_set /content/Dataset/Testing --weights weights.pt

**Comparison between original image and reconstructed image**

In [ ]:
path = '/content/Results/Input'
os.chdir(path)
inputs = os.listdir()
for item in inputs:
    if not item.endswith(".png"):
        os.remove(os.path.join(path, item))
inputs.sort()
path = '/content/Results/Reconstructed'
os.chdir(path)
outputs = os.listdir()
for item in outputs:
    if not item.endswith(".png"):
        os.remove(os.path.join(path, item))
outputs.sort()
os.chdir('/content')

In [ ]:
!mv /content/Binarized_VAE/Datasets/Screenshot\ 2022-06-15\ at\ 4.58.08\ PM.png /content/
os.rename('/content/Screenshot 2022-06-15 at 4.58.08 PM.png','/content/white.png')

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import json
from PIL import Image

white = cv2.imread('white.png')
resized_white = cv2.resize(white, (160,120), interpolation = cv2.INTER_AREA)
with open('/content/KL_Losses.json') as kl_json_file:
    kl_data = json.load(kl_json_file)
with open('/content/MSE_Losses.json') as mse_json_file:
    mse_data = json.load(mse_json_file)
for i in range(len(inputs)):
  input_path = os.path.join('/content/Results/Input',inputs[i])
  output_path = os.path.join('/content/Results/Reconstructed',outputs[i])
  img1 = cv2.imread(input_path)
  img2 = cv2.imread(output_path)
  #img = Image.fromarray(img2.astype('uint8'), 'RGB')
  #img.save(f'/content/Temp/{i}.jpg') 
  # scale_percent = 500 # percent of original size
  # width1 = int(img1.shape[1] * scale_percent / 100)
  # height1 = int(img1.shape[0] * scale_percent / 100)
  # dim1 = (width1, height1)
  # width2 = int(img2.shape[1] * scale_percent / 100)
  # height2 = int(img2.shape[0] * scale_percent / 100)
  # dim2 = (width2, height2)

  # resized1 = cv2.resize(img1, dim1, interpolation = cv2.INTER_AREA)
  # resized2 = cv2.resize(img2, dim2, interpolation = cv2.INTER_AREA)

  im_v = cv2.hconcat([img1, resized_white, img2])

  cv2_imshow(im_v)
  print("KL Loss : " + str(float(kl_data[inputs[i][:-4]][1:-1])))
  print("MSE Loss : " + str(float(mse_data[f'{int(inputs[i][:-4])}'][1:-1])))
  print()